In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import math
from graphviz import Digraph

In [2]:
#Load Iris dataset
iris = datasets.load_iris();
X = pd.DataFrame(iris.data, columns=iris.feature_names)
Y  = pd.DataFrame(iris.target)
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
# Class for each decision tree node
class Node:
    isLeaf = False
    #output
    #featureIndex
    #featureName
    #featureValue
    #leftChild
    #rightChild
    #impurity
    

In [6]:
class DecisionTrees:
    root = None
    impurity_measure = None
    
    def predictHelper(self, X, node):
        if(node.isLeaf == True):
            return node.output
        
        if(type(X[node.featureIndex]) == np.bool or type(X[node.featureIndex]) == np.str):
            if(X[node.featureIndex] == node.feature_value):
                return self.predictHelper(X[:node.featureIndex] + X[node.featureIndex+1:], node.leftChild)
            else:
                return self.predictHelper(X[:node.featureIndex] + X[node.featureIndex+1:], node.rightChild)
        else:
            if(X[node.featureIndex] < node.feature_value):
                return self.predictHelper(X[:node.featureIndex] + X[node.featureIndex+1:], node.leftChild)
            else:
                return self.predictHelper(X[:node.featureIndex] + X[node.featureIndex+1:], node.rightChild)
        
    def predict(self, X):
        return self.predictHelper(X, root)
    
    def entropy(self, Y): # Calculate entropy
        entropySum = 0
        unique_Y = Y.iloc[:,0].unique()
        if(len(unique_Y) == 1):
            return 0
        totalNumOfY = len(Y)
        for i in unique_Y:
            numOfi = (Y == i).sum()[0]
            #print("Count of ", i, " : ", numOfi)
            Pi = numOfi / totalNumOfY
            entropySum += -1* (Pi) * math.log(Pi, 2)

        return entropySum
    
    def gini_index(self, Y): # Calculate gini index
        giniIndex = 1
        unique_Y = Y.iloc[:,0].unique()
        totalNumOfY = len(Y)
        for i in unique_Y:
            numOfi = (Y == i).sum()[0]
            #print("Count of ", i, " : ", numOfi)
            Pi = numOfi / totalNumOfY
            giniIndex -= Pi*Pi

        return giniIndex
    
    def split(self, X, Y, feature_index, feature_value): # Split data for children
        remainingX, leftY, rightY = None, None, None
        remainingX = X.drop(X.columns[feature_index], axis=1)
        if(X.iloc[:,feature_index].dtype == np.bool or X.iloc[:,feature_index].dtype == np.str):
            leftY = Y.loc[ X.iloc[:, feature_index] == feature_value]
            rightY = Y.loc[ X.iloc[:, feature_index] != feature_value]
        else:
            leftY = Y.loc[ X.iloc[:, feature_index] < feature_value]
            rightY = Y.loc[ X.iloc[:, feature_index] > feature_value]
        
        return remainingX, leftY, rightY
            
            
    #Recursive function to create tree
    def __createTree(self, X, Y, root, heightOfNode=0):
        if ( len(Y.iloc[:,0].unique())==1 \
            or len(Y) < self.min_sample_split \
            or len(X.columns)==0):
            root.isLeaf = True
            root.output = Y.mode().iloc[0,0]
            print("Leaf Created with output : ", root.output, '\n')
            return

        best_feature_value = None
        min_impurity = 10
        best_feature_index = -1
        
        if(self.impurity_measure == "entropy"):
            root.impurity = self.entropy(Y)
        elif(self.impurity_measure == "gini_index"):
            root.impurity = self.gini_index(Y)
            
        print("Level", heightOfNode)
        unique_Y = Y.iloc[:,0].unique()
        for i in unique_Y:
            print("Count of ", i, " : ", (Y == i).sum()[0])
        print("Current", self.impurity_measure, " : ", root.impurity)
        
        # Iterate over each feature
        for i in range(X.shape[1]): 
            #If feature is discrete
            if(X.iloc[:,i].dtype == np.bool or X.iloc[:,i].dtype == np.str):
                distinct_feature_values = X.iloc[:, i].unique()
                n = len(distinct_feature_values)
                if(len(distinct_feature_values) == 2):
                    n = 1
                for j in range(n): #Iterate for each feature value
                    df1 = Y[X.iloc[:, i] == distinct_feature_values[j]]
                    df2 = Y[X.iloc[:, i] != distinct_feature_values[j]]
                    impurity1, impurity2 = None, None
                    if(self.impurity_measure == "entropy"):
                        impurity1 = self.entropy(df1)
                        impurity2 = self.entropy(df2)
                    elif(self.impurity_measure == "gini_index"):
                        impurity1 = self.gini_index(df1)
                        impurity2 = self.gini_index(df2)
                    
                    avg_impurity = (len(df1)/len(Y))*impurity1 + (len(df2)/len(Y))*impurity2
                    if(avg_impurity < min_impurity):
                        min_impurity = avg_impurity
                        best_feature_value = distinct_feature_values[j]
                        best_feature_index = i
            
            #Handle continuous attribute            
            else:
                distinct_feature_values = sorted(X.iloc[:, i].unique())
                #Iterate for each feature value
                for j in range(len(distinct_feature_values)-1): 
                    median = (distinct_feature_values[j] + distinct_feature_values[j+1])/2
                    df1 = Y.loc[X.iloc[:, i] < median]
                    df2 = Y.loc[X.iloc[:, i] > median]
                    
                    impurity1, impurity2 = None, None
                    if(self.impurity_measure == "entropy"):
                        impurity1 = self.entropy(df1)
                        impurity2 = self.entropy(df2)
                    elif(self.impurity_measure == "gini_index"):
                        impurity1 = self.gini_index(df1)
                        impurity2 = self.gini_index(df2)
                    
                    avg_impurity = (len(df1)/len(Y))*impurity1 + (len(df2)/len(Y))*impurity2
                    if(avg_impurity < min_impurity):
                        min_impurity = avg_impurity
                        best_feature_value = median
                        best_feature_index = i
        
        root.featureIndex = best_feature_index
        root.featureName = X.columns[best_feature_index]
        root.featureValue = best_feature_value
        print("Split Feature : ", X.columns[best_feature_index])
        print("Split Feature Value : ", best_feature_value, '\n')
        root.leftChild = Node()
        root.rightChild = Node()
        remainingX, leftY, rightY = self.split(X, Y, best_feature_index, best_feature_value)
        self.__createTree(remainingX, leftY, root.leftChild, heightOfNode+1)
        self.__createTree(remainingX, rightY, root.rightChild, heightOfNode+1)
    
    def fit(self, X, Y, impurity_measure, min_sample_split=1):
        self.root = Node()
        self.impurity_measure = impurity_measure
        self.min_sample_split = min_sample_split
        self.__createTree(X, Y, self.root)
    
    def createGraphHelper(self, dot, node, path):
        if(node.isLeaf):
            dot.node(path, str(node.output))
            return
        
        NodeContent = ""
        condition = node.featureName
        if(type(node.featureValue) == bool or type(node.featureValue) == str):
            condition += "=" + str(node.featureValue)
        else:
            condition += "<" + str(node.featureValue)
        NodeContent = condition + "\n"
        NodeContent += self.impurity_measure + " = " + str(node.impurity)
        dot.node(path, NodeContent)   
        self.createGraphHelper(dot, node.leftChild, path+"l")
        self.createGraphHelper(dot, node.rightChild, path+"r")
        dot.edge(path, path+"l","T") 
        dot.edge(path, path+"r", "F") 
    
    def createGraph(self, path):
        dot = Digraph(comment='Decision Tree')
        self.createGraphHelper(dot, self.root, "")
        dot.render(path, view=True)
        

In [7]:
#Instantiate a classifer
clf = DecisionTrees()

# fit(features, labels, measure of impurity to be used "entropy" or "gini_index", min_sample_split)
clf.fit(X, Y, "gini_index", min_sample_split = 2)

Level 0
Count of  0  :  50
Count of  1  :  50
Count of  2  :  50
Current gini_index  :  0.6666666666666665
Split Feature :  petal length (cm)
Split Feature Value :  2.45 

Leaf Created with output :  0 

Level 1
Count of  1  :  50
Count of  2  :  50
Current gini_index  :  0.5
Split Feature :  petal width (cm)
Split Feature Value :  1.75 

Level 2
Count of  1  :  49
Count of  2  :  5
Current gini_index  :  0.1680384087791495
Split Feature :  sepal length (cm)
Split Feature Value :  7.05 

Level 3
Count of  1  :  49
Count of  2  :  4
Current gini_index  :  0.13955144179423276
Split Feature :  sepal width (cm)
Split Feature Value :  2.8499999999999996 

Leaf Created with output :  1 

Leaf Created with output :  1 

Leaf Created with output :  2 

Level 2
Count of  1  :  1
Count of  2  :  45
Current gini_index  :  0.04253308128544431
Split Feature :  sepal length (cm)
Split Feature Value :  5.95 

Level 3
Count of  1  :  1
Count of  2  :  6
Current gini_index  :  0.24489795918367352
Split

In [6]:
#Create graph the trained decision tree
clf.createGraph("./Graph for Iris dataset")

In [8]:
# Data of OR gate
data = [[1,1], [1,0], [0,1], [0,0]]
X = pd.DataFrame(data, columns=['A', 'B'])
Y = pd.DataFrame([1,0,0,0])


In [9]:
clf2 = DecisionTrees()
clf2.fit(X, Y, "entropy")
clf2.createGraph("./Graph for OR gate")

Level 0
Count of  1  :  1
Count of  0  :  3
Current entropy  :  0.8112781244591328
Split Feature :  A
Split Feature Value :  0.5 

Leaf Created with output :  0 

Level 1
Count of  1  :  1
Count of  0  :  1
Current entropy  :  1.0
Split Feature :  B
Split Feature Value :  0.5 

Leaf Created with output :  0 

Leaf Created with output :  1 

